In [1]:
import copy, os, sys, gc, json
import numpy as np
import pickle as pkl
from time import time, sleep
from tqdm import tqdm

from IPython import display
from collections import Counter
from itertools import product, chain, combinations

#matplotlib
import matplotlib
import matplotlib as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
from matplotlib.colors import colorConverter

%matplotlib inline
#https://matplotlib.org/users/customizing.html
matplotlib.rcParams['legend.markerscale'] = 1.5     # the relative size of legend markers vs. original
matplotlib.rcParams['legend.handletextpad'] = 0.01
matplotlib.rcParams['legend.labelspacing'] = 0.4    # the vertical space between the legend entries in fraction of fontsize
matplotlib.rcParams['legend.borderpad'] = 0.5       # border whitespace in fontsize units
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['axes.titlesize'] = 20

matplotlib.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
matplotlib.rcParams['image.interpolation'] = 'nearest'
matplotlib.rcParams['image.cmap'] = 'gray'


# Configuring latex fonts
matplotlib.rc('font', **{'family':'serif'})
matplotlib.rc('text', usetex=True)
matplotlib.rc('text.latex', unicode=True)
matplotlib.rc('text.latex', preamble=r'\usepackage[utf8]{inputenc}')
matplotlib.rc('text.latex', preamble=r'\usepackage[english]{babel}')
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')

current_dir = !pwd
outer_dir, innder_dir = current_dir.get_nlstr().split('git')
library_path = os.path.join(outer_dir, 'git/ml')
if library_path not in sys.path:
    sys.path.append(library_path)


from ml.neural_network import Layer, Sequential, Model, Solver
from ml.neural_network.layers import *
from ml.neural_network.nonlinearities import *
from ml.neural_network.criterions import MSECriterion, MulticlassLogLoss
from ml.neural_network.regularizers import *
from ml.neural_network.initializers import *
from ml.neural_network.optimizers import *
from ml.neural_network.decorators import *
from ml.neural_network.gradients import *
from ml.neural_network.dataset import Dataset, CocoDataset
from ml.utils.image_utils import image_from_url

run the following from the cs231n directory and try again:
python setup.py build_ext --inplace
You may also need to restart your iPython kernel


# Table of contents
* [Tensor](#tensor)
* [Variable](#variable)
* [Constant](#constant)

# Tensor [[toc]](#toc)

<a id='tensor_shape'></a>
## TensorShape [[toc]](#toc)

In [54]:
class TensorShape:
    def __init__(self, shape=None):
        self.set_shape(shape)
    def set_shape(self, shape):
        self.initialized = False
        if shape is None:
            self.shape = None
            self.initialized = False
        elif isinstance(shape, (tuple, list)):
            self.shape = tuple(shape)
            self.initialized = True
        elif isinstance(shape, TensorShape):
            self.shape = shape.get_shape() # так как хранится в immutable tuple-ах, то проблем быть не должно
            self.initialized = True
        else:
            self.initialized = False
            raise TypeError('Unacceptable type "{}" of "shape"'.format(type(shape)))
    def get_shape(self):
        return self.shape
    def is_initialized(self):
        return self.initialized
    def __repr__(self):
        return type(self).__name__ + '(' + repr(self.shape) + ')'
    def __len__(self):
        return len(self.shape)
    def __getitem__(self, n):
        return self.shape[n]
    def __iter__(self):
        return iter(self.shape)
    def __eq__(self, other_shape):
        other_shape = TensorShape(other_shape)
        # Если хотя бы одна из форм не инициализована, то результат сравнения False
        if (not self.is_initialized()) | (not other_shape.is_initialized()):
            return False
        if len(self.shape) != len(other_shape):
            return False
        for l, r in zip(self.shape, other_shape):
            if (l == -1) | (r == -1):
                continue
            if l != r:
                return False
        return True
    def __neq__(self, other_shape):
        return not self == outher_shape

In [122]:
class Tensor:
    """
    Класс Tensor служит для хранения значения некоторой переменной
    """
    def __init__(self, value=None, shape=None, name=None, dtype=None, validate_shape=True):
        """
        -dtype: если тип dtype не задан, то
            либо наследуется от value, если тот имеет аттрибут dtype;
            либо полагается равным np.float64
        -name:  если имя не задано, то
            либо наследуется от value, в случае наличия у value аттрибута name;
            либо полагается равным имени типа, т.е. в данном случае Tensor
        -validate_shape: проверка последующих изменений
        """
        self.set_name(value, name)
        self.set_dtype(value, dtype)
        self.set_value(value)
        self.validate_shape = validate_shape

    def set_name(self, value, name):
        self._type_name = type(self).__name__
        if name is None:
            if hasattr(value, 'name'):
                name = name
            else:
                name = self._type_name
        if not isinstance(name, str):
            raise TypeError('{}.__init__: name must be string.'.format(self._type_name))
        self.name = name
    def set_dtype(self, value, dtype):
        if dtype is None:
            if hasattr(value, 'dtype'):
                dtype = value.dtype
            else:
                dtype = np.float64
        self.dtype = dtype
        
    def set_value(self, value):
        if isinstance(value, Tensor):
            value = value.value
        self.value = np.array(value, copy=False, dtype=self.dtype)
        self.shape = TensorShape(self.value.shape)
        
    def __repr__(self):
        return '{}\n[name={}, dtype={}\nshape={}\nvalue=\n{}\n]'.format(
            self._type_name, self.name, self.dtype, self.shape, self.value)

    def get_value(self):
        return self.value
    
    def get_shape(self):
        return self.shape
    
    def __abs__(self):
        op_name = 'abs(' + self.name + ')'
        return Tensor(np.abs(self.value), dtype=self.dtype, name=op_name)
        
    def __add__(self, right):
        assert isinstance(right, Tensor)
        op_name = self.name + '+' + right.name
        return Tensor(self.value + right.value, dtype=self.dtype, name=op_name)
        
    def __bool__(self):
        raise TypeError('{} cannot be treated as bool.'.format(type(self).__name__))
        
    def dot(self, right):
        if not isinstance(right, Tensor):
            raise TypeError('Right argument of {}.dot must be a Tensor.'.format(self._type_name))
        if self.value.ndim != 2 or right.value.ndim != 2:
            raise ValueError('{}.dot: values must be a 2-dim arrays but their shapes are {} and {}.'.format(
                self._type_name, self.get_shape(), right.get_shape()))
        if self.get_shape()[1] != right.get_shape()[0]:
            raise ValueError('{}.dot: inconsistent shapes {} and {} of arguments.'.format(
                self._type_name, self.get_shape(), right.get_shape()))
        op_name = self.name + '.dot(' + right.name + ')'
        return Tensor(np.dot(self.value, right.value), dtype=self.dtype, name=op_name)
    
    def multiply(self, right):
        if not isinstance(right, Tensor):
            raise TypeError('{}.multiply: right argument must be a Tensor.'.format(self._type_name))
        if self.get_shape() != right.get_shape():
            raise ValueError('{}.multiply: inconsistent shapes {} and {} of arguments.'.format(
                self._type_name, self.get_shape(), right.get_shape()))
        op_name = self.name + '.multiply(' + right.name + ')'
        return Tensor(np.multiply(self.value, right.value), dtype=self.dtype, name=op_name)
    
    def __mul__(self, right):
        op_name = self.name + '*' + type(right).__name__
        return Tensor(self.value * right, dtype=self.dtype, name=op_name)
    
    def __rmul__(self, left):
        op_name = type(left).__name__ + '*' + self.name
        return Tensor(left * self.value, dtype=self.dtype, name=op_name)
    
    def __getitem__(self, slice_index):
        return self.value.__getitem__(slice_index)
    
    def __setitem__(self, slice_index, value):
        self.value[slice_index] = value

In [123]:
np.random.seed(5)
t1 = Tensor(np.random.randn(4, 5), name='t1')
t2 = Tensor(np.random.randn(5, 2))
print(t1)
print(t2)
print(t1.shape, t2.shape)
print(t1.dot(t2))
print(2 * t1)
print(t2 * 2)
print(t2[:, 0])
t2[:, 0] = 1
print(t2)
#print(t1.multiply(t2))

Tensor
[name=t1, dtype=float64
shape=TensorShape((4, 5))
value=
[[ 0.44122749 -0.33087015  2.43077119 -0.25209213  0.10960984]
 [ 1.58248112 -0.9092324  -0.59163666  0.18760323 -0.32986996]
 [-1.19276461 -0.20487651 -0.35882895  0.6034716  -1.66478853]
 [-0.70017904  1.15139101  1.85733101 -1.51117956  0.64484751]]
]
Tensor
[name=Tensor, dtype=float64
shape=TensorShape((5, 2))
value=
[[-0.98060789 -0.85685315]
 [-0.87187918 -0.42250793]
 [ 0.99643983  0.71242127]
 [ 0.05914424 -0.36331088]
 [ 0.00328884 -0.10593044]]
]
TensorShape((4, 5)) TensorShape((5, 2))
Tensor
[name=t1.dot(Tensor), dtype=float64
shape=TensorShape((4, 2))
value=
[[ 2.26337556  1.57343799]
 [-1.33857222 -1.4265056 ]
 [ 1.02092714  0.81005268]
 [ 1.44618905  1.91739989]]
]
Tensor
[name=int*t1, dtype=float64
shape=TensorShape((4, 5))
value=
[[ 0.88245497 -0.6617403   4.86154237 -0.50418426  0.21921968]
 [ 3.16496223 -1.81846481 -1.18327332  0.37520645 -0.65973992]
 [-2.38552922 -0.40975302 -0.71765789  1.20694321 -3.3

In [124]:
Tensor(3, 't1') + Tensor(2, 't2')

Tensor
[name=t1+t2, dtype=<class 'numpy.float64'>
shape=TensorShape(())
value=
5.0
]

In [ ]:
def get_variable(name, shape=None, dtype=np.float64, initializer=None):
    """
    -shape:
    -dtype:
    
    """
    if isinstance(initializer, )
    
    return initializer(shape)

In [ ]:
class Tensor:
    """
    Прокси класс для передачи доступа к параметрам
    
    
    """
    def __init__(self, shape=None, init_value=None, initializer=None, dtype=np.float64, name=None):
        """
        - shape: tuple, list или TensorShape
        - value: numpy array
        """
        self.initialized = False
        shape = TensorShape(shape)
        if init_value is not None:
            assert isinstance(init_value, np.ndarray)

        if shape.is_initialized() & (init_value is not None):
            init_value_shape = TensorShape(init_value.shape)
            if shape != init_value_shape:
                raise ValueError('Inconsistent parameters "shape" and "init_value" parameters '\
                                 'are passed to the {} constructor "{}": {} != {}.'.format(
                                 type(self).__name__, shape, init_value_shape))
            self.shape = init_value_shape  
            self.initializer = ConstantInitializer(value=init_value, dtype=dtype)
        elif shape.is_initialized() & (init_value is None):
            if initializer is None:
                # Используем инициализатор по умолчанию (плохая практика)
                initializer = NormalInitializer(dtype=dtype)
            if not isinstance(initializer, InitializerBase):
                raise TypeError('Parameter "initializer" passed to the {} constructor '\
                                'must have type Initializer.'.format(type(self).__name__))
            self.shape = shape
            self.initializer = initializer
        elif (not self.shape.is_initialized()) & (init_value is not None):
            self.shape = TensorShape(init_value.shape)
            self.initializer = ConstantInitializer(value=init_value, dtype=dtype)
        else:
            raise ValueError('Either "shape" or "init_value" must be provided to the {} constructor.'.format(type(self).__name__))
        self.name = name
        self.dtype = dtype
        assert isinstance(self.initializer, InitializerBase)
        assert self.shape.is_initialized()
        for dim_size in self.shape:
            assert dim_size > 0, 'All "shape" dimensions must be positive integers.'

    def __repr__(self):
        if self.initialized:
            return 'Tensor[shape={}, value=\n{}]'.format(self.shape, self.value)
        return 'Tensor[shape={}, value=None]'.format(self.shape)
    
    def initialize(self):
        """
        Инициализация аргумента value данного тензора.
        Для возможности инициализации необходимо выполнение одного из следующих условий:
        - задано начальное значение value тензора; доложно быть совместимо с параметром shape
        - задано значение shape формы тензора
        """
        self.value = self.initializer(shape=self.shape.get_shape())
        self.initialized = True
    def get_value(self):
        return self.value


class Placeholder:
    def __init__(self, shape):
    
    def get_value():
    
    def set_value(self, value):
        assert self.shape == TensorShape(value.shape)
    
    
        
class TensorConstantInitializer(ConstantInitializer):
    def __init__(self, name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name
    def __call__(self, *args, **kwargs):
        tensor_value = super().__call__(*args, **kwargs)
        return Tensor(shape=tensor_value.shape, init_value=tensor_value, name=self.name,
                      dtype=self.dtype)

    
class TensorNormalInitializer(NormalInitializer):
    def __init__(self, name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.name = name
    def __call__(self, *args, **kwargs):
        tensor_value = super().__call__(*args, **kwargs)
        return Tensor(shape=tensor_value.shape, init_value=tensor_value, name=self.name,
                      dtype=self.dtype)

In [ ]:
class Dense(Layer):
    def __init__(self, units, use_bias=True, W_init=None, b_init=None, W_reg=None, b_reg=None):
        """
        Inputs:
        - units - Integer or Long, dimensionality of the output space.
        - W_initializer
        - b_initializer
        - seed - used for initializers!!!
        """
        super().__init__()
        self.set_units(units)
        self.set_use_bias(use_bias)
        self.W_init = W_init
        self.b_init = b_init
        self.W_reg  = W_reg
        self.b_reg  = b_reg
    def set_units(self, units):
        self._check_int_positive(units, 'units', '"units" must be a positive integer.')
        self.units = units
    def set_use_bias(self, use_bias):
        self._check_boolean(use_bias, 'use_bias', '"use_bias" must be a boolean')
        self.use_bias = use_bias
    def __repr__(self):
        input_size = -1
        if self.initialized:
            input_size = self.W.shape[0]
        return 'Dense({}->{})'.format(input_size, self.units)   
   
    ################################## 
    ###       Initialization       ###
    ##################################
    def _initialize_input_shape(self, params):
        super()._initialize_input_shape(params)
        assert len(self.input_shape) == 2, 'input to Dense layer must be a 2-dim tensor.'
        self.n_features = self.input_shape[1]
        return params
    def _initialize_params(self, params):
        self._initialize_W(params)
        self._initialize_b(params)
        return params
    def _initialize_W(self, params):
        W_name = self.get_param_name('W')
        W_shape = (self.n_features, self.units)
        W_init = self.W_init
        if W_init is None:
            W_init = NormalInitializer(generator=self.generator, dtype=self.dtype, loc=0.0, scale=1.0 / np.sqrt(self.n_features))
        W_init = get_variable(name=W_name, shape=W_shape, dtype=self.dtype, validate_shape=True, initializer=W_init)
        self.W = W_init()
        self.grad_W = Tensor(value=self.W.get_zeros_like(), name=W_name, validate_shape=True)
        if self.W_reg is None:
            self.W_reg = EmptyRegularizer()
        assert isinstance(self.W, Tensor):
        assert isinstance(self.grad_W, Tensor)
        assert isinstance(self.W_reg, Regularizer), 'W_reg of layer "{}" must have type Regularizer.'.format(self.name)
        self._params[W_name] = self.W
        self._grad_params[W_name] = self.grad_W
        self._regularizers[W_name] = self.W_reg
        return params
    def _initialize_b(self, params):
        b_name = self.get_param_name('b')
        b_shape = (self.units,)
        b_init = self.b_init
        if b_init is None:
            b_init = ConstantInitializer(dtype=self.dtype)
        b_init = get_variable(name=b_name, shape=b_shape, dype=self.dtype, validate_shape=True, initializer=b_init)
        self.b = b_init()
        self.grad_b = Tensor(value=sel.b.get_zeros_like(), name=b_name, validate_shape=True)
        if self.b_reg is None:
            self.b_reg = EmptyRegularizer()
        assert isinstance(self.b, Tensor)
        assert isinstance(self.grad_b, Tensor)
        assert isinstance(self.b_reg, Regularizer), 'b_reg of layer "{}" must have type Regularizer.'.format(self.name)
        self._params[b_name] = self.b
        self._grad_params[b_name] = self.grad_b
        self._regularizers[b_name] = self.b_reg
        return params  
    
    def _initialize_output_shape(self, params):
        self.output_shape = (-1, self.units) # Input shape for the next layer
        params['input_shape'] = self.output_shape
        return params
    
    def set_W(self, W):
        self.W.set_value(W)
    def set_b(self, b):
        self.b.set_value(b)
        
    ################################## 
    ###     Forward propagation    ###
    ##################################
    # Проверки прямого распространения
    def _forward_check_shape(self, input, target=None):
        assert input.ndim == 2, 'Input to layer "{}" must be 2-dim numpy array'.format(self.name)
        assert input.shape[1] == self.W.shape[0], 'Expected input shape (-1, {}) but received (-1, {})'.format(self.W.shape[0], input.shape[1])
    # Прямое распространение
    def _forward(self, input, target=None):
        self.output = np.dot(input, self.W)  # [N x D] x [D x H] = [N x H]
        if self.use_bias:
            self.output += self.b[None, :]
    
    ################################## 
    ###    Backward propagation    ###
    ##################################
    # Проверка ошибок обратного распространения
    def _backward_check_shape(self, input, grad_output):
        assert input.ndim == 2
        assert grad_output.ndim == 2
        assert input.shape[1] == self.W.shape[0]
        assert input.shape[0] == grad_output.shape[0]
        assert grad_output.shape[1] == self.W.shape[1]
    # Обратное распространение
    def _backward(self, input, grad_output):
        self.grad_input = np.dot(grad_output, self.W.T)          # [N x H] x [H x D] = [N x D]
        self.grad_W = np.dot(input.T, grad_output)               # ([D x N] x [N x H]).T = [D, H]
        self.W_reg.update_grad(self.W, self.grad_W)
        if self.use_bias:
            self.grad_b = np.sum(grad_output, axis=0)
            self.b_reg.update_grad(self.b, self.grad_b)

    ################################## 
    ###         Parameters         ###
    ##################################
    @check_initialized
    def get_params(self, copy=False):  
        params = OrderedDict()
        params[self.name + ':W'] = self.W 
        params[self.name + ':b'] = self.b
        return self._make_dict_copy(params, copy=copy)
    @check_initialized
    def get_grad_params(self, copy=False):
        grad_params = OrderedDict()
        grad_params[self.name + ':W'] = self.grad_W
        grad_params[self.name + ':b'] = self.grad_b
        return self._make_dict_copy(grad_params, copy=copy)
        
    ################################## 
    ###       Regularization       ###
    ##################################
    @check_initialized
    def get_regularizers(self):
        regularizers = OrderedDict()
        regularizers[self.name + ':W'] = self.W_reg
        regularizers[self.name + ':b'] = self.b_reg
        return regularizers